In [1]:
from huggingface_hub import login

login()

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# 모델 및 토크나이저 불러오기
model_name = "Qwen/Qwen2.5-Math-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

# 저장할 숫자 리스트 (0 ~ 10)
numbers = [str(i) for i in range(11)]  # 숫자를 문자열로 변환하여 리스트 생성

# 숫자별 임베딩 벡터 저장 딕셔너리
embeddings_dict = {}

# 숫자별 임베딩 추출
for num in numbers:
    # 토큰 변환
    inputs = tokenizer(num, return_tensors="pt")

    # 모델에 입력하여 임베딩 추출
    with torch.no_grad():
        outputs = model(**inputs)

    # 첫 번째 토큰의 히든 상태 벡터 사용
    embedding = outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()

    # 숫자를 문자열 키로 사용하여 저장
    embeddings_dict[num] = embedding

# NumPy 파일로 저장 (NPZ 형식)
np.savez("qwen2.5_math_embeddings.npz", **embeddings_dict)

# 저장된 데이터 확인
loaded_data = np.load("qwen2.5_math_embeddings.npz")

# 결과 출력 예시 (숫자 0과 1의 임베딩 확인)
print("0 EMBEDDING:", loaded_data["0"])
print("1 EMBEDDING:", loaded_data["1"])


0 EMBEDDING: [-0.3932364  -1.2312586   0.3884799  ...  0.43546918 -0.81787926
  0.87561166]
1 EMBEDDING: [-0.343621   -1.1170435   0.09171043 ...  1.1479254  -0.24104886
  0.7149127 ]
